In [ ]:
!pip install yfinance

In [ ]:
import numpy as np
import pandas as pd

import yfinance as yf
import pandas_datareader as pdr

import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

import matplotlib.pyplot as plt


Question 1: IPO Filings Web Scraping and Data Processing
What's the total sum ($m) of 2023 filings that happened on Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [ ]:
import pandas as pd
import requests
from datetime import datetime

url="https://stockanalysis.com/ipos/filings/"
response=requests.get(url)
ipo_dfs=pd.read_html(response.text)[0]
ipo_dfs=ipo_dfs.rename(columns={'Filing Date': 'Filing_Date','Price Range':'Price_Range'})
ipo_dfs

,Filing_Date,Symbol,Company Name,Price Range,Shares Offered
0,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
1,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
2,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
3,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
4,"Apr 22, 2024",DRJT,Derun Group Inc,$5.00,-
...,...,...,...,...,...
320,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
321,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
322,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
323,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [ ]:
ipo_dfs['Filing_Date'] = pd.to_datetime(ipo_dfs['Filing_Date'])

ipo_dfs = ipo_dfs[ipo_dfs['Filing_Date'].dt.year == 2023]
ipo_dfs.head()
df=ipo_dfs
df.head()

<ipython-input-45-42718276429f>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Filing_Date,Symbol,Company Name,Price Range,Shares Offered
49,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000
50,2023-12-29,EPSM,Epsium Enterprise Limited,-,-
51,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667
52,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000
53,2023-12-22,LZMH,LZ Technology Holdings Limited,-,-


In [ ]:

df['Shares Offered'] = pd.to_numeric(df['Shares Offered'].str.replace('-',''),errors='coerce')
df['Shares Offered']=df['Shares Offered'].astype('float')

In [ ]:
df.head()

,Filing_Date,Symbol,Company Name,Price Range,Shares Offered
49,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0
50,2023-12-29,EPSM,Epsium Enterprise Limited,-,NaN
51,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667.0
52,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000.0
53,2023-12-22,LZMH,LZ Technology Holdings Limited,-,NaN


In [ ]:
def Price_Range(input_string):
    input_split = input_string.split('-')
    if len(input_split) == 1:
        return float(input_split[0].replace('$', '').strip())
    else:
        lp=input_split[0].replace('$', '').strip()
        up=input_split[1].replace('$', '').strip()
        if lp=='' or up=='':
            return None
        lower_price = float(lp)
        upper_price = float(up)
        return (lower_price + upper_price) / 2

df['Avg_price'] = df.apply(lambda x: Price_Range(x['Price Range']), axis=1)
print(df.head())


   Filing_Date Symbol                    Company Name    Price Range  \
49  2023-12-29    LEC           Lafayette Energy Corp  $3.50 - $4.50   
50  2023-12-29   EPSM       Epsium Enterprise Limited              -   
51  2023-12-28   ONDR       Sushi Ginza Onodera, Inc.  $7.00 - $8.00   
52  2023-12-27   JDZG                   Jiade Limited  $4.00 - $5.00   
53  2023-12-22   LZMH  LZ Technology Holdings Limited              -   

    Shares Offered  Avg_price  
49       1200000.0        4.0  
50             NaN        NaN  
51       1066667.0        7.5  
52       2200000.0        4.5  
53             NaN        NaN  


In [ ]:
df['Shared_offered_value'] = df['Shares Offered'] * df['Avg_price']
print(df.head())

   Filing_Date Symbol                    Company Name    Price Range  \
49  2023-12-29    LEC           Lafayette Energy Corp  $3.50 - $4.50   
50  2023-12-29   EPSM       Epsium Enterprise Limited              -   
51  2023-12-28   ONDR       Sushi Ginza Onodera, Inc.  $7.00 - $8.00   
52  2023-12-27   JDZG                   Jiade Limited  $4.00 - $5.00   
53  2023-12-22   LZMH  LZ Technology Holdings Limited              -   

    Shares Offered  Avg_price  Shared_offered_value  
49       1200000.0        4.0             4800000.0  
50             NaN        NaN                   NaN  
51       1066667.0        7.5             8000002.5  
52       2200000.0        4.5             9900000.0  
53             NaN        NaN                   NaN  


In [ ]:
print(df['Shared_offered_value'][df['Filing_Date'].dt.dayofweek==4].sum())

285700000.0


In [ ]:
print(len(df['Shared_offered_value'][df['Filing_Date'].dt.dayofweek==4]))

32


Question 2: IPOs "Fixed days hold" strategy
Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this link. Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Additional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [ ]:
url1='https://stockanalysis.com/ipos/2023/'

response=requests.get(url1)
ipo_df2023=pd.read_html(response.text)[0]

ipo_df2023.head()





,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.04,0.40%
1,"Dec 19, 2023",LGCB,Linkage Global Inc,$4.00,$3.10,-22.50%
2,"Dec 15, 2023",ZKH,ZKH Group Limited,$15.50,$12.34,-20.39%
3,"Dec 15, 2023",BAYA,Bayview Acquisition Corp,$10.00,$10.17,1.70%
4,"Dec 14, 2023",INHD,Inno Holdings Inc.,$4.00,$0.66,-83.40%


In [ ]:
url2='https://stockanalysis.com/ipos/2024/'
response=requests.get(url2)
ipo_df2024=pd.read_html(response.text)[0]


ipo_df2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$29.00,20.83%
1,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.17,-20.75%
2,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$32.75,2.34%
3,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$48.16,72.00%
4,"Apr 25, 2024",MRX,Marex Group plc,$19.00,$18.93,-0.37%


In [ ]:
ipo_df=pd.concat([ipo_df2023,ipo_df2024],ignore_index=True)

In [ ]:
ipo_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.04,0.40%
1,"Dec 19, 2023",LGCB,Linkage Global Inc,$4.00,$3.10,-22.50%
2,"Dec 15, 2023",ZKH,ZKH Group Limited,$15.50,$12.34,-20.39%
3,"Dec 15, 2023",BAYA,Bayview Acquisition Corp,$10.00,$10.17,1.70%
4,"Dec 14, 2023",INHD,Inno Holdings Inc.,$4.00,$0.66,-83.40%


In [ ]:
print(len(ipo_df['Symbol'].unique()))

218


In [ ]:
ipo_df['IPO Date'] = pd.to_datetime(ipo_df['IPO Date'])

In [ ]:
ipo_df=ipo_df[ipo_df['IPO Date']<datetime(2024,3,1)]

In [ ]:
print(len(ipo_df['Symbol'].unique()))

185


In [ ]:
ipo_df=ipo_df[ipo_df['Symbol']!='RYZB']
print(ipo_df.head())

    IPO Date Symbol                   Company Name IPO Price Current   Return
0 2023-12-27   IROH  Iron Horse Acquisitions Corp.    $10.00  $10.04    0.40%
1 2023-12-19   LGCB             Linkage Global Inc     $4.00   $3.10  -22.50%
2 2023-12-15    ZKH              ZKH Group Limited    $15.50  $12.34  -20.39%
3 2023-12-15   BAYA       Bayview Acquisition Corp    $10.00  $10.17    1.70%
4 2023-12-14   INHD             Inno Holdings Inc.     $4.00   $0.66  -83.40%


In [ ]:
print(ipo_df['Symbol'].values)

['IROH' 'LGCB' 'ZKH' 'BAYA' 'INHD' 'AFJK' 'GSIW' 'FEBO' 'CLBR' 'ELAB' 'RR'
 'DDC' 'SHIM' 'GLAC' 'SGN' 'HG' 'CRGX' 'ANSC' 'AITR' 'GVH' 'LXEO' 'PAPL'
 'ATGL' 'MNR' 'WBUY' 'NCL' 'BIRK' 'GMM' 'PMEC' 'LRHC' 'GPAK' 'SPKL' 'QETA'
 'MSS' 'ANL' 'SYRA' 'VSME' 'LRE' 'TURB' 'MDBH' 'KVYO' 'CART' 'DTCK' 'NMRA'
 'ARM' 'SPPL' 'NWGL' 'SWIN' 'IVP' 'NNAG' 'SRM' 'SPGC' 'LQR' 'NRXS' 'FTEL'
 'MIRA' 'PXDT' 'CTNT' 'HRYU' 'SRFM' 'PRZO' 'HYAC' 'KVAC' 'JNVR' 'ELWS'
 'WRNT' 'TSBX' 'ODD' 'APGE' 'NETD' 'SGMT' 'BOWN' 'SXTP' 'PWM' 'VTMX'
 'INTS' 'SVV' 'KGS' 'FIHL' 'GENK' 'BUJA' 'BOF' 'AZTR' 'CAVA' 'ESHA' 'ATMU'
 'ATS' 'IPXX' 'CWD' 'SGE' 'SLRN' 'ALCY' 'KVUE' 'GODN' 'TRNR' 'AACT' 'JYD'
 'USGO' 'UCAR' 'WLGS' 'TPET' 'TCJH' 'GDTC' 'VCIG' 'GDHG' 'ARBB' 'ISPR'
 'MGIH' 'MWG' 'HSHP' 'SFWL' 'SYT' 'HKIT' 'CHSN' 'TBMC' 'HLP' 'ZJYL' 'TMTC'
 'YGFGF' 'OAKU' 'BANL' 'OMH' 'MGRX' 'FORL' 'ICG' 'IZM' 'AESI' 'AIXI'
 'SBXC' 'BMR' 'DIST' 'GXAI' 'MARX' 'BFRG' 'ENLT' 'MLYS' 'PTHR' 'BLAC'
 'NXT' 'HSAI' 'LSDI' 'LICN' 'GPCR' 'ASST' 'CETU' 'TXO'

In [ ]:
url3='https://stockanalysis.com/actions/changes/'
data=requests.get(url3)
df_tick_change=pd.read_html(data.text)[0]
df_tick_change['Date']=pd.to_datetime(df_tick_change['Date'])

df_tick_change=df_tick_change[(df_tick_change['Date']<datetime(2024,3,1)) & (df_tick_change['Date']>=datetime(2023,1,1))]
df_tick_change
print(df_tick_change.head())

         Date   Old    New          New Company Name
47 2024-02-29   POL  POLCQ           Polishedcom Inc
48 2024-02-28  SZZL   CRML      Critical Metals Corp
49 2024-02-27   ROI   ROII  Riskon International Inc
50 2024-02-27  TMST   MTUS              Metallus Inc
51 2024-02-26  AMEH   ASTH        Astrana Health Inc


In [ ]:
ticker_dict=dict(zip(df_tick_change.Old,df_tick_change.New))

In [ ]:
ipo_df['Symbol'] = ipo_df['Symbol'].apply(lambda x: ticker_dict[x] if x in ticker_dict else x)


In [ ]:
import yfinance as yf

def growth(df):
    grow_df=pd.DataFrame()
    for i in range(1, 30):
        grow_df['growth_'+str(i)] = df['Adj Close'].shift(i) / df['Adj Close']
    return grow_df

items = ipo_df['Symbol']
newdf = pd.DataFrame()

for item in items:
    df = yf.download(tickers=item, period='max', interval='1d')
    growthdf = growth(df)  # Call the growth function to calculate growth columns
    if len(newdf)==0:
          newdf=growthdf
    else:
          newdf = pd.concat([growthdf])

newdf.head()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,growth_1,growth_2,growth_3,growth_4,growth_5,growth_6,growth_7,growth_8,growth_9,growth_10,...,growth_20,growth_21,growth_22,growth_23,growth_24,growth_25,growth_26,growth_27,growth_28,growth_29
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-10,0.734177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-11,0.977723,0.717822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-12,1.002481,0.980149,0.719603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-16,1.030691,1.033248,1.010230,0.741688,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
a=newdf.describe()

In [ ]:
a

,growth_1,growth_2,growth_3,growth_4,growth_5,growth_6,growth_7,growth_8,growth_9,growth_10,...,growth_20,growth_21,growth_22,growth_23,growth_24,growth_25,growth_26,growth_27,growth_28,growth_29
count,80.000000,79.000000,78.000000,77.000000,76.000000,75.000000,74.000000,73.000000,72.000000,71.000000,...,61.000000,60.000000,59.000000,58.000000,57.000000,56.000000,55.000000,54.000000,53.000000,52.000000
mean,1.039160,1.077784,1.123001,1.173295,1.215001,1.229171,1.247640,1.265413,1.281043,1.300278,...,1.336864,1.329407,1.329283,1.329367,1.334040,1.328262,1.318691,1.325431,1.335949,1.361079
std,0.293479,0.388114,0.492004,0.618104,0.686985,0.715266,0.740769,0.766988,0.799987,0.864961,...,0.746251,0.709380,0.671923,0.656872,0.637406,0.587752,0.537918,0.508537,0.507078,0.563104
min,0.661765,0.604027,0.603937,0.641732,0.590164,0.536364,0.569930,0.547857,0.501307,0.419126,...,0.552206,0.504027,0.480537,0.522148,0.542424,0.589394,0.628452,0.599163,0.577419,0.577419
25%,0.969083,0.943449,0.934242,0.943548,0.938075,0.984496,0.973774,0.975443,0.971249,0.978379,...,0.874016,0.842438,0.892784,0.885903,0.859504,0.795436,0.939163,0.935589,0.982759,0.871705
50%,1.010070,1.027811,1.031248,1.030000,1.062258,1.091837,1.110672,1.096774,1.103659,1.131250,...,1.286408,1.307303,1.333333,1.336202,1.320755,1.313411,1.315068,1.349199,1.330749,1.373567
75%,1.074792,1.117353,1.150853,1.156643,1.198858,1.208738,1.239587,1.274272,1.287089,1.272769,...,1.426573,1.443433,1.479970,1.477182,1.475090,1.528143,1.545238,1.585594,1.620000,1.589022
max,3.429825,3.535088,3.543860,4.296703,4.428572,4.439560,4.344086,4.330011,4.627219,5.025707,...,3.960784,3.872549,3.731482,3.740741,3.657407,3.181102,3.110236,2.821429,2.792857,3.370690


In [ ]:
max_values = a.max(axis=1)
max_columns = a.idxmax(axis=1)

# Combine into a DataFrame
result = pd.DataFrame({'Max Value': max_values, 'Column Name': max_columns})

print(result)

       Max Value Column Name
count  80.000000    growth_1
mean    1.376010   growth_16
std     1.031561   growth_14
min     0.661765    growth_1
25%     1.001406   growth_11
50%     1.373567   growth_29
75%     1.620000   growth_28
max     5.642458   growth_13


Question 3: Is Growth Concentrated in the Largest Stocks?
Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

Now let's add the top 12-22 stocks (as of end-April 2024):

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:

Date	ticker_category	growth_7d
2014-01-01	LARGE	1.011684
2014-01-01	LARGEST	1.011797
On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [ ]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS


In [ ]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime
def growth_7d(df):
    grow_df=pd.DataFrame()
    grow_df['growth_7d'] = df['Adj Close'] / df['Adj Close'].shift(-7)
    return grow_df

items = LARGEST_STOCKS
newdf = pd.DataFrame()
newdf.index=pd.date_range('2014-01-01','2023-12-31')
for item in LARGEST_STOCKS:
    df = yf.download(tickers=item, period='max', interval='1d')
    df = df.loc['2013-12-24':'2023-12-31']
    growth_df = growth_7d(df) # Call the growth function to calculate growth columns
    newdf[item]=growth_df

newdf.head()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,MSFT,AAPL,GOOG,NVDA,AMZN,META,BRK-B,LLY,AVGO,V,...,TCS.NS,HDB,BHARTIARTL.NS,IBN,SBIN.NS,LICI.NS,INFY,ITC.NS,HINDUNILVR.NS,LT.NS
2014-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.944057,NaN,1.021782,NaN,1.097565,NaN,NaN,0.999380,1.041015,1.117672
2014-01-02,1.062322,1.032479,0.991220,1.032552,1.017878,0.978537,1.030611,0.974266,1.000190,1.009639,...,0.914828,0.990907,0.993052,0.988740,1.061752,NaN,0.934691,0.967737,1.030603,1.056601
2014-01-03,1.031582,0.990099,0.961371,0.989268,0.997233,0.944926,1.022704,0.974447,0.956215,0.993308,...,0.955066,0.990616,1.004269,1.004163,1.056621,NaN,0.944160,0.970103,1.049405,1.041204
2014-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
newdf = newdf.mean(axis=1).dropna()

In [ ]:
newdf.head(20)

2014-01-01    1.036741
2014-01-02    1.008135
2014-01-03    1.001436
2014-01-06    0.988404
2014-01-07    0.993609
2014-01-08    0.997885
2014-01-09    0.983092
2014-01-10    0.986427
2014-01-13    0.990716
2014-01-14    1.013677
2014-01-15    1.033813
2014-01-16    1.031566
2014-01-17    1.033428
2014-01-20    1.025062
2014-01-21    1.036434
2014-01-22    1.035152
2014-01-23    1.046303
2014-01-24    1.018291
2014-01-27    1.008938
2014-01-28    0.998526
dtype: float64

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime
def growth_7d(df):
    grow_df=pd.DataFrame()
    grow_df['growth_7d'] = df['Adj Close'] / df['Adj Close'].shift(-7)
    return grow_df

newdf_large_stock = pd.DataFrame()
newdf_large_stock.index=pd.date_range('2014-01-01','2023-12-31')
for item in LARGE_STOCKS:
    df = yf.download(tickers=item, period='max', interval='1d')
    df = df.loc['2013-12-24':'2023-12-31']
    growth_df = growth_7d(df) # Call the growth function to calculate growth columns
    newdf_large_stock[item]=growth_df

newdf_large_stock.head()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,PRX.AS,CDI.PA,AIR.PA,SU.PA,ETN,SNY,BUD,DTE.DE,ALV.DE,MDT,...,BAJFINANCE.NS,MARUTI.NS,HCLTECH.NS,TATAMOTORS.NS,SUNPHARMA.NS,ONGC.NS,ADANIENT.NS,NTPC.NS,KOTAKBANK.NS,TITAN.NS
2014-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.006248,0.983223,0.969420,1.021662,0.939203,1.014258,1.115377,1.051750,1.031551,1.047900
2014-01-02,NaN,1.033905,1.013804,0.984348,1.019171,1.034379,1.009949,0.993855,0.997276,0.966076,...,1.002107,0.991951,0.931291,0.989898,0.952685,0.961762,1.085211,1.016196,0.986748,1.023745
2014-01-03,NaN,1.045420,1.005348,0.987011,1.000132,1.025217,1.007869,0.987923,0.988408,0.981990,...,1.001578,1.014239,0.953634,0.975524,0.960484,0.950215,1.016786,1.003809,1.001757,1.028959
2014-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
newdf_large_stock = newdf_large_stock.mean(axis=1).dropna()


In [ ]:
a=(newdf < newdf_large_stock).sum()/len(newdf)

In [ ]:
a

0.5463678516228748

import pandas aQuestion 4: Trying Another Technical Indicators strategy
What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years? One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
are you still profitable on those trades?

In [ ]:
# https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab
# Update (apr 2023): Colab is now Python 3.10
# there are some other (older) ways to install talib

url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4087    0  4087    0     0  12066      0 --:--:-- --:--:-- --:--:-- 12091
100  517k  100  517k    0     0   651k      0 --:--:-- --:--:-- --:--:-- 3424k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4079    0  4079    0     0  10950      0 --:--:-- --:--:-- --:--:-- 10994
100  392k  100  392k    0     0   455k      0 --:--:-- --:--:-- --:--:-- 2824k


In [ ]:
import pandas as pd


def talib_get_momentum_indicators_for_one_ticker(df: pd.DataFrame) -> pd.DataFrame:

  # CCI - Commodity Channel Index
  talib_momentum_cci = talib.CCI(df.High.values, df.Low.values, df.Close.values,timeperiod=14)
  df['cci']=talib_momentum_cci
  return df




In [ ]:
from datetime import datetime,timedelta

items = LARGEST_STOCKS
newdf = pd.DataFrame()
newdf.index=pd.date_range('2014-01-01','2023-12-31')
# newdf = newdf[~newdf.index.day_name().isin(['Saturday','Sunday'])]
print(len(newdf))
for item in [LARGEST_STOCKS[0]]:

    print(item)
    df = yf.download(tickers=item, period='max', interval='1d')
    df = df.loc['2013-12-10':'2023-12-31']
    # df = df[~df.index.day_name().isin(['Saturday','Sunday'])]
    a=talib_get_momentum_indicators_for_one_ticker(df)
    a = a.loc['2014-01-01':'2023-12-31']
    a = a[a.index.day_name().isin(['Friday'])]
    trade_date =a[a['cci']>200].index
    sum+=a[trade_date,'Adj Close'] - a[trade_date+timedelta(days=5),'Adj Close']




3652
MSFT


[*********************100%%**********************]  1 of 1 completed
<ipython-input-48-bd46e521a844>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cci']=talib_momentum_cci


InvalidIndexError: (DatetimeIndex(['2014-05-30', '2014-09-05', '2015-04-24', '2015-10-02',
               '2015-10-23', '2016-01-29', '2016-10-21', '2017-01-27',
               '2017-04-21', '2017-10-27', '2018-01-05', '2018-06-01',
               '2019-02-22', '2021-07-23', '2021-08-13', '2021-08-20',
               '2022-07-29', '2023-03-17', '2023-04-28'],
              dtype='datetime64[ns]', name='Date', freq=None), 'Adj Close')

MSFT


[*********************100%%**********************]  1 of 1 completed


AttributeError: 'NoneType' object has no attribute 'loc'

In [51]:
import pandas as pd
import yfinance as yf
from talib import abstract
from datetime import timedelta

import pandas as pd


def talib_get_momentum_indicators_for_one_ticker(df: pd.DataFrame) -> pd.DataFrame:

  # CCI - Commodity Channel Index
  talib_momentum_cci = talib.CCI(df.High.values, df.Low.values, df.Close.values,timeperiod=14)
  df['cci']=talib_momentum_cci
  return df

# Initialize an empty DataFrame
newdf = pd.DataFrame()
newdf.index = pd.date_range('2014-01-01', '2023-12-31')
total_price_diff=0
# Iterate over each stock in LARGEST_STOCKS (just taking the first one in your example)
for item in LARGEST_STOCKS:

    # Download stock data using yfinance
    df = yf.download(tickers=item, period='max', interval='1d')
    df['Date']=df.index
    df.index=range(len(df))
    df = df.loc[(df['Date']>='2013-12-10') & (df['Date']<='2024-1-5')]

    # df=df.apply(pd.to_numeric).info()
    # Calculate momentum indicators
    a = talib_get_momentum_indicators_for_one_ticker(df)

    a = a.loc[(a['Date']>='2014-01-01') & (a['Date']<='2023-12-31')]
    # Filter Fridays with CCI > 200
    a = a[(a.Date.dt.dayofweek == 4) & (a['cci'] > 200)]
    # Calculate the difference between closing prices on Fridays and five days later
    trade_dates = a.index
    df['Adj Close']=df['Adj Close'].astype('float')

    # print(df.loc[trade_dates+timedelta(days=5), 'Adj Close'])
    price_diff = df.loc[trade_dates+5, 'Adj Close'].values - df.loc[trade_dates, 'Adj Close'].values
    print(price_diff)
    # Sum up the price differences
    price = price_diff.sum()

    total_price_diff+=price

print("Total price difference:", total_price_diff)


[*********************100%%**********************]  1 of 1 completed
<ipython-input-51-67a4436dbf48>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cci']=talib_momentum_cci
<ipython-input-51-67a4436dbf48>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Adj Close']=df['Adj Close'].astype('float')


[ 0.46051788  0.67789078  0.68703079  1.3568573  -0.20264816 -4.37302017
  0.19011307 -1.91386795  1.88879776  0.30593109  1.31372833  0.78973389
  1.48496246 -4.64831543 11.80886841 -4.5397644   2.13644409  1.1307373
  3.36242676]


[*********************100%%**********************]  1 of 1 completed
<ipython-input-51-67a4436dbf48>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cci']=talib_momentum_cci
<ipython-input-51-67a4436dbf48>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Adj Close']=df['Adj Close'].astype('float')


[ 0.64928627 -0.29842758  0.09505653  0.46937561  0.28824234  0.34745026
  0.06374359  1.784729    4.9413681   0.42861938 -6.97770691 -0.30610657
 -0.75727844 -5.33892822]


[*********************100%%**********************]  1 of 1 completed
<ipython-input-51-67a4436dbf48>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cci']=talib_momentum_cci
<ipython-input-51-67a4436dbf48>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Adj Close']=df['Adj Close'].astype('float')


...

[*********************100%%**********************]  1 of 1 completed
<ipython-input-51-67a4436dbf48>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cci']=talib_momentum_cci
<ipython-input-51-67a4436dbf48>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Adj Close']=df['Adj Close'].astype('float')


[  30.92352295   66.3973999    -1.92443848   14.71728516    1.9173584
   38.48474121  104.82543945  -33.00439453   56.42114258 -114.60107422
  -21.07788086   14.0847168   -12.40698242   17.7734375   -41.34985352]


[*********************100%%**********************]  1 of 1 completed

[ 69.26916504  -2.78137207  -8.37042236  -2.13305664  36.42785645
  -4.96325684 -20.60241699 -86.34716797  -2.5612793   39.9597168
  22.69873047 -26.14086914  63.15771484   6.94995117 187.80004883]
Total price difference: 303.4902458190918



<ipython-input-51-67a4436dbf48>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cci']=talib_momentum_cci
<ipython-input-51-67a4436dbf48>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Adj Close']=df['Adj Close'].astype('float')


In [52]:
print("Total price difference:", total_price_diff)

Total price difference: 303.4902458190918


In [21]:
import pandas as pd
import yfinance as yf
from talib import abstract, CCI
from datetime import timedelta

# Define the list of largest stocks (LARGEST_STOCKS) - Not provided in the snippet

def talib_get_momentum_indicators_for_one_ticker(df: pd.DataFrame) -> pd.DataFrame:
    # CCI - Commodity Channel Index
    talib_momentum_cci = CCI(df.High.values, df.Low.values, df.Close.values)
    df['cci'] = talib_momentum_cci
    return df

# Initialize an empty DataFrame
total_gross_profit = 0

# Iterate over each stock in LARGEST_STOCKS
for item in LARGEST_STOCKS:
    # Download stock data using yfinance
    df = yf.download(tickers=item, period='max', interval='1d')
    df = df.loc['2014-01-01':'2023-12-31']

    # Calculate momentum indicators
    df = talib_get_momentum_indicators_for_one_ticker(df)

    # Filter Fridays with CCI > 200
    a = df[(df.index.dayofweek == 4) & (df['cci'] > 200)]

    # Calculate the difference between closing prices on Fridays and five days later
    trade_dates = a.index
    price_diff = df.loc[trade_dates, 'Adj Close'].shift(-5) - df.loc[trade_dates, 'Adj Close']

    # Calculate gross profit for each trade
    profit_per_trade = (price_diff * 1000).sum()  # Assuming investment of $1000 per trade

    # Accumulate total gross profit
    total_gross_profit += profit_per_trade

# Convert total gross profit to thousands of dollars
total_gross_profit_thousands = round(total_gross_profit / 1000)

print("Total gross profit in THOUSANDS $:", total_gross_profit_thousands)


[*********************100%%**********************]  1 of 1 completed
<ipython-input-21-78684764f189>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cci'] = talib_momentum_cci
[*********************100%%**********************]  1 of 1 completed
<ipython-input-21-78684764f189>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cci'] = talib_momentum_cci
[*********************100%%**********************]  1 of 1 completed
<ipython-input-21-78684764f189>:11: SettingWithCopyWarning: 
A value is trying 

Total gross profit in THOUSANDS $: 61528



<ipython-input-21-78684764f189>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cci'] = talib_momentum_cci
